In [1]:
# from downloaded pages, extract features

In [2]:
import requests
from bs4 import BeautifulSoup
import pprint
import pymysql as mdb
import re
import pandas as pd
import numpy as np
from unidecode import unidecode

In [3]:
con = mdb.connect('localhost','root','hobbes','kickstarter');

In [12]:
# how many rows do I have?
with con:
    cur = con.cursor()
    cur.execute("SELECT COUNT(Id) FROM Scraped_pages")
    rows = cur.fetchall()
    
    print rows[0]
    cur.close()

(76994,)


In [63]:
with con:
    cur = con.cursor()

    cur.execute("SELECT URL from Project_info WHERE Id = 9000")
    rows = cur.fetchall()
    cur.close()

In [64]:
print rows

(('https://www.kickstarter.com/projects/wave/tampa-co-working-only-you-can-end-high-tech-homele?ref=category',),)


In [69]:
def get_creator_activity(creator_text_only):
    # initialize for -1 so I can see where my parsing failed
    n_backed = -1
    n_created = -1
    n_comments = -1
    
    # get number of campaigns backed
    match = re.search(r'Backed\n\((\d+)\)',creator_text_only)
    if match:
        n_backed = match.group(1)
        
    #match = re.search('is not backing any projects',creator_text_only)
    
    # get number of campaigns created
    match = re.search(r'Created\n\((\d+)\)',creator_text_only)
    if match:
        n_created = match.group(1)
        
    # get number of comments
    match = re.search(r'Comments\n\((\d+)\)',creator_text_only)
    if match:
        n_comments = match.group(1)
    
    return n_backed, n_created, n_comments
    

In [75]:
with con:
    cur = con.cursor()

    cur.execute("SELECT Creator_html from Scraped_pages WHERE Id = 10002")
    retrieved_text = cur.fetchall()
    cur.close()

In [77]:
creator_text = BeautifulSoup(retrieved_text[0][0])
creator_text_only = creator_text.get_text()
print creator_text_only




  //<![CDATA[
    docElement = this.document.documentElement;
    docElement.className=docElement.className.replace(/\bno-js\b/,'') + 'js';
  //]]>

The NoPhone Team — Kickstarter
  //<![CDATA[
    window.report_errors = true;
    window.honeybadger_configuration = {
      api_key : "02ab109335c0dd1a93506d92f22217a7",
      environment : "production",
      component : "users",
      action : "show",
      onerror : false
    };
  //]]>



window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"b04b883ad2","applicationID":"11086","transactionName":"cFteRUULX1wHFhtNEVZGQx5EDFxH","queueTime":4,"applicationTime":122,"ttGuid":"","agentToken":null,"agent":"js-agent.newrelic.com/nr-632.min.js","userAttributes":"SEk="}
(window.NREUM||(NREUM={})).loader_config={xpid:"Vw4BVEVSCQMIUg=="};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];

In [78]:
#backing seems broken, but still "is not backing any projects"
get_creator_activity(creator_text_only)

(u'0', u'2', u'0')

In [79]:
with con:
    cur = con.cursor()

    cur.execute("SELECT Project_html from Scraped_pages WHERE Id = 12000")
    retrieved_text = cur.fetchall()
    cur.close()

In [80]:
this_soup = BeautifulSoup(retrieved_text[0][0])
project_text = this_soup.get_text()

In [81]:
print project_text








About this project



iPhone 5 Support for LabStrip & First Sci-Fi Sensor


Labstrip/Anywhere Science Cloud Demo







Evolution of a Design




Our goal when we started this project roughly one year ago was to automate the interpretation of chemical test strips using the power of today's smartphones.  Our LabStrip Meter design began simply as an idea made real through Lego blocks.  Those early models helped us figure out the basic meter geometries and critical design inputs.  It became clear that it was critical to control the light source when imaging the strip, so we moved to an enclosed box and augmented the iPhone's flash.  The design then moved to hand-made models of foamboard and plastic.  We then solicited the help of the professionals at Speck Design of Palo Alto, CA who are well known for their iOS device accessory designs.  They came up with an elegant core industrial design.  Source One Product Development of Murrieta, CA then translated Speck's industrial model int

In [126]:
# I'm only extracting two digit rewards. why?

In [94]:
def extract_reward_level(project_text):
    # extract reward levels!
    matches = re.findall('\$(\d+) reward',project_text)
    #for match in matches:
    #    print match
    
    n_rewards = len(matches)
    matches_str = str(matches)
    
    return n_rewards, matches_str

In [95]:
extract_reward_level(project_text)

(11,
 "[u'5', u'10', u'35', u'45', u'50', u'60', u'100', u'110', u'130', u'200', u'375']")

In [88]:
print a

[u'5', u'10', u'35', u'45', u'50', u'60', u'100', u'110', u'130', u'200', u'375']


In [93]:
# Video -- Play and 00:00?

In [94]:
# features --
#regular expression $. reward -- occurances? 
# Contains "Ships anywhere in the world"
# Video -- "Play", whitespace


In [ ]:
# save n rewards.
# I need to save this as a strong csv

In [73]:
print matches

[u'15', u'20', u'25']


In [119]:
# to extract
reward_levels = [int(m) for m in matches]
print reward_levels

[20, 20, 20, 25, 25, 25, 40, 55]


In [133]:
# where did I get kicked off?
with con:
    cur = con.cursor()
    
    
    cur.execute("SELECT COUNT(Id) FROM Scraped_pages")
    rows = cur.fetchall()
    print rows
    cur.close()

((9350,),)


In [134]:
# now look at a food example
with con:
    cur = con.cursor()

    cur.execute("SELECT Creator_html, Project_html FROM\
        Scraped_pages WHERE Id = 93522")
    retrieved_text = cur.fetchall()
    cur.close()

In [135]:
np.size(retrieved_text)

2

In [136]:
this_soup = BeautifulSoup(retrieved_text[0][0])
creator_text = this_soup.get_text()

this_soup = BeautifulSoup(retrieved_text[0][1])
project_text = this_soup.get_text()

In [139]:
print project_text








About this project







You'll need an HTML5 capable browser to see this content.





Play








00:00


00:00


































Artisan Kits featuring NYC's Finest Spices from Kalustyan's



New York, NY

Food





$6,561


pledged of $6,000 goal










The best things in life should be enjoyed by and accessible to everyone. It is with this in mind that our company strives to create lasting, one-of-a-kind, artisanal products that showcase quality ingredients, and then share them with you. 









All of the spices featured in our full and micro kits are sourced directly from Kalustyan's spice emporium in the heart of NYC. Vetted, chosen, and beloved by the culinary elite of New York City, Kalustyan's sources the finest ingredients from over 70 countries to distribute at their Murray Hill storefront and at select Whole Foods markets. And now, with our help, we're bringing them to you.





Our goal is to craft a variety of kits that cater to every level